# Advanced examples with list comprehensions and generators

This is an extract of Fluent Python [Chap. 2 examples](https://github.com/fluentpython/example-code-2e/blob/master/02-array-seq/array-seq.ipynb)

# Bugs due to Immutable objects storing mutable objects
Bugs can occur it I stuck in an immutable ojects (like a tuple) a mutable one, like a list. You can use the hash function to check if you are not writing vulnerable code.

In [1]:
# This tuple that contains mutable objects and may lead to bugs
t = ([1,2,3], 'this', 'is', 'mutable', 'stuff')
# You can check the mutability of the objects insude it, using the hash function
for x in t:
    try:
        hash(x)
    except TypeError:
        print('The tuple contains a mutable object')
        break

# In realty, you can check this more quickly using hash on th tuple itself. 
# If the container is immutable, and only contains immutable objects, you'll be able to get a hash key.
print('This tuple only contains immutable objects, so is hashable')
t = (1, 'this', 'is', 'immutable', 'stuff')
hash(t)

The tuple contains a mutable object
This tuple only contains immutable objects, so is hashable


1850231896817634060

# Walrus operator `:=`

In [2]:
# Walrus operator :=
# When a variable is assigned using this operator, it remains accessible outside the scope of the list comprehension
x = '$%^£@'
codes = [last := ord(c) for c in x]
last

64

# List comprehension can be seen as a "shortcut" of `map` and `filter` 

See the example below. Note that in situations `map` and `filter` together may be faster. Speed test!

In [3]:
codes_filtered = list(
    filter(
        lambda c: c > 120, 
        map(ord, x) # creates a generator that applies `ord`` to all elements of x 
))
codes_filtered


[163]

# Generator expressions to build tuples and arrays
To build arrays/tuples you could start from list comprehension and convert to tuple/array. But you can save memory by using generator expressions, which yields an item at time through the iterator protocol.

- A generator expression is **always** in parenthesis `(...)` unless they are the **single** argument of a function.

In [4]:
t = tuple(ord(c) for c in x)
print(t)
import array 
a = array.array('I', (ord(c) for c in x))
print(a)

(36, 37, 94, 163, 64)
array('I', [36, 37, 94, 163, 64])


# `*` for Packing/Unpacking

`*anything` unpacks any number of arguments.

In [5]:
a, *other, b, c = 1, 'not', 'important', 'arguments', 2, 3
print(other)

['not', 'important', 'arguments']


It can not be used in functions, though - unless you stick to the usual convention of adding keyword arguments after `*args, **kwargs` format. For call below, only works if the last arguments are passed as keywords:

In [6]:
def f(a, b, *args, c, d):
    print(f'args={args}')
    print(f'a={a}, b={b}, c={c}, d={d}')
f(1, 2, 0, 0, 0, c=3, d=4)
try:
    f(1, 2, 0, 0, 0, 3, 4)
except TypeError:
    print('''f(1, 2, 0, 0, 0, 3, 4) -> `TypeError: f() missing 2 required keyword-only arguments: 'c' and 'd'`''')

args=(0, 0, 0)
a=1, b=2, c=3, d=4
f(1, 2, 0, 0, 0, 3, 4) -> `TypeError: f() missing 2 required keyword-only arguments: 'c' and 'd'`


# Slices

Slices are objects. They are only 1D, but the `[]` operator can take multiple slices at time.

In [7]:
all_ids = slice(0,4,1)
name_ids = slice(2,4,1)
L = ['group_id', 'personal_id', 'name', 'surname']
print(L[all_ids])
print(L[name_ids])


['group_id', 'personal_id', 'name', 'surname']
['name', 'surname']


# Memoryview

Allows sharing slices of arrays (and other flat sequences; not lists of tuple) without copying the bytes. This memory can be shared also to other objects. Let's see an example:

In [8]:
from array import array
a = array('b', range(6))
# arrays share memory by default - nothing special here
b = a 
b[0] = 99
print(a)
# however, once I cast into a list, memory is not shared anymore
L = a.tolist()
a[0] = 100 
print(L)
# unless I use a memory view
mview = memoryview(a)
mview.tolist()
L = mview.cast('b',[2,3])
L[1,1] = 40 
print(a)
print('L is a bit obscure...')
print(L)

array('b', [99, 1, 2, 3, 4, 5])
[99, 1, 2, 3, 4, 5]
array('b', [100, 1, 2, 3, 40, 5])
L is a bit obscure...
